In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_764682/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [8]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse_sup", ascending=True, ignore_index=True)
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [9]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_4_8_6,0.999723,0.731521,0.997848,0.999488,0.999156,0.000136,0.159381,0.000389,0.000280,0.000335,0.003090,0.011661,1.000021,0.011754,3535.806151,8524.826025,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
1,model_4_9_2,0.999723,0.728430,0.999812,0.999343,0.999603,0.000136,0.161216,0.000104,0.000294,0.000199,0.003311,0.011661,1.000021,0.011755,3535.805929,8524.825803,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
2,model_4_8_7,0.999723,0.731269,0.997719,0.999447,0.999098,0.000136,0.159531,0.000412,0.000303,0.000358,0.003046,0.011671,1.000021,0.011764,3535.802695,8524.822569,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
3,model_4_8_5,0.999722,0.731760,0.997995,0.999530,0.999219,0.000137,0.159239,0.000362,0.000257,0.000310,0.003141,0.011694,1.000021,0.011788,3535.794761,8524.814635,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
4,model_4_8_8,0.999720,0.731034,0.997598,0.999405,0.999042,0.000138,0.159670,0.000434,0.000326,0.000380,0.003012,0.011729,1.000021,0.011823,3535.782765,8524.802639,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,model_6_2_21,0.990714,0.704738,0.965795,0.980913,0.973644,0.004562,0.175280,0.019221,0.011033,0.015127,0.012106,0.067544,1.000710,0.068087,3528.779900,8517.799775,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
621,model_6_2_22,0.990713,0.704753,0.965801,0.980899,0.973640,0.004563,0.175272,0.019218,0.011041,0.015130,0.012106,0.067549,1.000710,0.068092,3528.779601,8517.799475,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
622,model_6_2_18,0.990709,0.704662,0.965709,0.980965,0.973628,0.004565,0.175326,0.019270,0.011003,0.015136,0.012140,0.067565,1.000711,0.068107,3528.778683,8517.798557,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
623,model_6_2_23,0.990694,0.704752,0.965724,0.980867,0.973586,0.004572,0.175272,0.019261,0.011060,0.015161,0.012110,0.067619,1.000712,0.068162,3528.775485,8517.795359,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
